# Setup

To run the example, a file named `secret.py` must be created in the notebooks
folder with the following content:

```python
# Configuration settings

# SLIPO workbench installation
BASE_URL = 'https://app.dev.slipo.eu'

# SLIPO API key
API_KEY = ''
```

The `API_KEY` value must be set to a valid SLIPO Application Key. The file must be imported before creating a new context:

```python
from secret import BASE_URL, API_KEY
```

In [ ]:
# Create new context
from slipoframes.context import SlipoContext

from secret import BASE_URL, API_KEY

ctx = SlipoContext(
    base_url = BASE_URL,
    requires_ssl = False,
    api_key = API_KEY
)

# Transform operation

Next we are going to:

* Upload the files `DKV_Fuel_Berlin.csv` and `OSM_Fuel_Berlin.csv` from the local folder `datasets` to the remote folder `notebooks/datasets`. The remote folder will be created automatically if not already exists. The option `overwrite` is also set to `True` to overwrite any existing files.
* Upload the contents of folder `config` from the local file system to the remote folder `notebooks/config`.
* Execute two transform operations to convert the `CSV` data to `N-Triples`.
* Check the status of each operation

In [ ]:
# Upload file DKV_Berlin.csv
ctx.file_upload('./datasets/DKV_Fuel_Berlin.csv', 'notebooks/datasets/DKV_Fuel_Berlin.csv', overwrite=True)

In [ ]:
# Upload file DKV_Berlin.csv
ctx.file_upload('./datasets/OSM_Fuel_Berlin.csv', 'notebooks/datasets/OSM_Fuel_Berlin.csv', overwrite=True)

In [ ]:
# Upload all files in the config folder
ctx.file_upload('./config', 'notebooks/config', overwrite=True)

In [ ]:
# Browse remote user file system
df_files = ctx.file_browse(sort_col='size', format_size=True, sort_asc=False)

df_files[df_files['path'].str.startswith("notebooks")]

In [ ]:
# Convert DKV_Fuel_Berlin.csv file to N-Triples
transform1 = ctx.transform_csv(
    'notebooks/datasets/DKV_Fuel_Berlin.csv',
    attrCategory='all_tags',
    attrKey='ID',
    attrName='name',
    attrX='lon',
    attrY='lat',
    delimiter=';',
    featureSource='DKV',
    quote='',
    mappingSpec='notebooks/config/DKV_Fuel_Berlin_slipo_mappings.yml',
    classificationSpec='notebooks/config/DKV_POI_sample_classification.csv'
)

In [ ]:
# Convert OSM_Fuel_Berlin.csv file to N-Triples
transform2 = ctx.transform_csv(
    'notebooks/datasets/OSM_Fuel_Berlin.csv',
    attrCategory='type',
    attrGeometry='wkt',
    attrKey='osm_id',
    attrName='name',
    attrX='lon',
    attrY='lat',
    delimiter='|',
    featureSource='OpenStreetMap',
    profile='OSM_Europe',
    quote='',
    mappingSpec='notebooks/config/OSM_Fuel_Berlin_slipo_mappings.yml',
    classificationSpec='notebooks/config/OSM_POI_sample_classification.csv'
)

In [ ]:
# Check process status for transform operations
transform1 = ctx.process_status(transform1)
transform2 = ctx.process_status(transform2)

# Interlink operation

Execute an interlink operation on the RDF datasets generated by the previous two transformation operations

In [ ]:
interlink1 = ctx.interlink(
    'SLIPO_MatchByName',
    left=transform1.output(),
    right=transform2.output()
)

In [ ]:
# Check process status for interlink operation
interlink1 = ctx.process_status(interlink1)

# Fuse Operation

Fuse the two RDF datasets generated by operations `transform1` and `transform2` using the links from operation `interlink`

In [ ]:
fuse1 = ctx.fuse(
    'SLIPO_default_abMode',
    left=transform1.output(),
    right=transform2.output(),
    links=interlink1.output()
)

In [ ]:
# Check process status for fuse operation
fuse1 = ctx.process_status(fuse1)

# Enrich Operation

Enrich the fused RDF dataset from operation `fuse1`

In [ ]:
enrich1 = ctx.enrich(
    'SLIPO_DEFAULT',
    source=fuse1.output()
)

In [ ]:
# Check process status for enrich operation
enrich1 = ctx.process_status(enrich1)

# Export Operation

Export the enriched RDF dataset to a CSV file

In [ ]:
export1 = ctx.export_csv(
    'SLIPO_default',
    enrich1.output(),
    delimiter='|',
    quote='"'
)

In [ ]:
# Check process status for export operation
export1 = ctx.process_status(export1)

In [ ]:
# Copy output file to local file system
ctx.process_file_download(export1.output(), target='./output/exported-data.zip', overwrite=True)